In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [ ]:
import cv2
import torch
import os
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import torchvision
import shutil
import imageio
from IPython import display
from PIL import Image
from glob import glob
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid, save_image

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Load Data

In [ ]:
root_dir = "/kaggle/input/anime-faces/data"

In [ ]:
os.makedirs('all_images')

In [ ]:
for file in os.listdir(os.path.join(root_dir)):
    if file.endswith('.png'):
        shutil.copy2(os.path.join(root_dir, file), os.path.join('all_images', file))

In [ ]:
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

In [ ]:
class AnimeFaces(Dataset):
    def __init__(self, folder):
        super().__init__()
        self.folder = folder
        self.images = sorted(glob(folder + '/*'))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, ix):
        image_path = self.images[ix]
        image = Image.open(image_path)
        image = transform(image)
        return image

In [ ]:
data = AnimeFaces(folder='./all_images')

In [ ]:
batch_size = 128
dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)

# Visualization

In [ ]:
data_iter = iter(dataloader)
images = next(data_iter)

img_grid = make_grid(images, nrow=16)

plt.figure(figsize=(8, 8))
plt.imshow(img_grid.permute(1, 2, 0))
plt.axis('off')
plt.show()

# Generator Network

In [ ]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.generator = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, output_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.generator(x)

In [ ]:
z_dim = 100
image_dim = 32 * 32 * 3

In [ ]:
generator = Generator(input_dim=z_dim, output_dim=image_dim).to(device)

# Discriminator Network

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.discriminator = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.discriminator(x)

In [ ]:
discriminator = Discriminator(input_dim=image_dim).to(device)

# Train

In [ ]:
lr = 0.0005
generator_optimizer = optim.RMSprop(generator.parameters(), lr=lr)
discriminator_optimizer = optim.RMSprop(discriminator.parameters(), lr=lr)

In [ ]:
d_loss_s = []
g_loss_s = []

In [ ]:
os.makedirs('generated-faces')

In [ ]:
n_epochs = 200

In [ ]:
for epoch in range(n_epochs):
    for batch_idx, real_images in enumerate(dataloader):
        real_images = real_images.view(-1, image_dim).to(device)
        batch_size = real_images.size(0)

        discriminator.zero_grad()

        z = torch.randn(batch_size, z_dim).to(device)
        fake_images = generator(z)

        real_output = discriminator(real_images)
        fake_output = discriminator(fake_images.detach())
        d_loss = -(torch.mean(real_output) - torch.mean(fake_output))
        d_loss_s.append(d_loss.item())

        d_loss.backward()
        discriminator_optimizer.step()

        for p in discriminator.parameters():
            p.data.clamp_(-0.01, 0.01)

        if batch_idx % 2 == 0:
            generator.zero_grad()
            
            z = torch.randn(batch_size, z_dim).to(device)
            fake_images = generator(z)

            fake_output = discriminator(fake_images)
            g_loss = -torch.mean(fake_output)
            g_loss_s.append(g_loss.item())

            g_loss.backward()
            generator_optimizer.step()

    print(f"Epoch [{epoch}/{n_epochs}], Loss_D: {d_loss.item():.4f}, Loss_G: {g_loss.item():.4f}")

    test_z = torch.randn(16, z_dim).to(device)
    generated = generator(test_z)
    save_image(generated.view(generated.size(0), 3, 32, 32), f'./generated-faces/sample_images_{epoch}.png')

# Results

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(g_loss_s)
plt.plot(d_loss_s)
plt.title("Loss Curve")
plt.legend(["Generator Loss", "Discriminator Loss"])
plt.show()

# GIF

In [ ]:
files = glob("./generated-faces/*.png")
gif_file = "./generated-faces/wgan_anime.gif"

In [ ]:
with imageio.get_writer(gif_file, mode='I') as writer:
    for file in files:
        image = imageio.imread(file)
        writer.append_data(image)

In [ ]:
display.Image(filename=gif_file)